Goal
Given: "54+7",
predict: "61".

In [5]:
import numpy as np
import tensorflow
tensorflow.config.run_functions_eagerly(True)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from termcolor import colored

Generate Data

In [6]:
all_chars = '0123456789+'

In [7]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    example = str(first_num) + '+' + str(second_num)
    label = str(first_num+second_num)
    return example, label

generate_data()

('36+26', '62')

Create Model

In [8]:
num_features = len(all_chars)

In [9]:
hidden_units = 128
max_time_steps = 5

model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               17920     
                                                                 
 repeat_vector (RepeatVecto  (None, 5, 128)            0         
 r)                                                              
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 5, 128)            32896     
                                                                 
 time_distributed (TimeDist  (None, 5, 11)             1419      
 ributed)                                                        
                                                                 
Total params: 52235 (204.04 KB)
Trainable params: 52235 (204.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Task 4: Vectorize and De-Vectorize Data

In [10]:
num_features = len(all_chars)

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))

print('Number of features:', num_features)

Number of features: 11


Vectorize & Devectorize Functions

In [11]:
def vectorize_example(example, label):

    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))

    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)

    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1

    return x, y

In [12]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

Create Dataset

In [13]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, max_time_steps, num_features))
    y_train = np.zeros((num_examples, max_time_steps, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y

    return x_train, y_train

x_train, y_train = create_dataset()
print(x_train.shape, y_train.shape)

(2000, 5, 11) (2000, 5, 11)


Train w/ early stopping

In [15]:
es_cb = EarlyStopping(monitor='val_loss', patience=10)
model.fit(x_train, y_train, epochs=500, batch_size=256, validation_split=0.2,
  verbose=False, callbacks=[es_cb])

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Evaluate Accuracy

In [34]:
x_test, y_test = create_dataset(250)
preds = model.predict(x_test)

3/8 [==========>...................] - ETA: 0s

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 [==============================] - 0s 33ms/step


In [62]:
pairs = zip(list(map(devectorize_example, preds)), list(map(devectorize_example,y_test)))
sum([1 if x==y else 0 for (x,y) in pairs])/len(preds)

0.772